论文标题 ： Electric Vehicle Routing Problem for Emergency Power Supply: Towards Telecom Base Station Relief∗

摘要部分 ： 

论文解决的问题 ： 电动汽车通过移动到基站的位置直接向基站供电，目标是想要同时最小化电动车总行驶距离，完全断电的数量

论文创新点 ： ev不仅要考虑行驶能耗和充电，还要考虑在基站的放电过程，总的操作周期 ： 移动 -》 准备 -》 充电/放电 -》清理

论文核心算法 ： 基于规则的车辆选择器，选择最早可以开始“移动”的电动车
               基于RL的节点选择器，使用Transformer架构，通过reinforce算法进行梯度训练

论文结果 ： 比贪婪分配，随机分配，基于时空网络分配的约束规划等基线相比，在目标值和计算时间上表现更优
           有良好的泛化能力，能够处理大规模问题

引言部分 ： 每个基站都有一个备用电池，用于短时间备份，但是需要外部的电池来充电维持长时间使用基站
            车辆选择器和节点选择器交替进行，同时，基于RL的节点选择即使在大规模问题中也能快速生成路由，这在紧急情况下至关重要
            数据集涉及基站，充电站和电动汽车的实际位置和规模，结果表明最终的模型优于基线（两个贪婪的方法和传统的启发式），同时，对求解器的泛化和可拓展性进行了分析，展示了对未知环境和大规模问题的适应能力

问题设定 ：1. 目标函数 ：分为两部分，用加号连接，第一部分是所有电动车的平均行驶距离，第二部分是α乘最小化断电基站比例，α是权重系数用于平衡两个目标的相对重要性
          2. 运动空间 ： 移动 和三个子动作 ：准备 -》 放电/充电 -》 清理 电动汽车在给定时间范围内维持这种循环
          3. 基站电池波动模型 ：分为两种情况，一种是无电动车供电，一种是有电动车供电
          4. 电动车电池波动模型 ：四种状态 1. 车辆移动 2. 车辆放电 3.车辆充电 4.其他
          5. 在不同状态下的时间计算 ：四种状态 1. 电动车移动状态 ：两个位置距离/电动车行驶 2.电动车充电 ： （β * 电池容量 -开始充电的电量）/充电速率 3.电动车放电 ： min(基站冲到目标电量的时间，电动车放电到安全下限的时间) 4. 电动车准备/清理 :固定时间
          6. 其他约束 ：基站电池约束 ： 0 《 基站电池 《 电量上限 ；电动车电量安全保障约束 ：从当前位置到目标位置再到充电站所需电量 《 当前电量 《 电量上限


方法论/模型 ：
    1.车辆选择器 ：遵循“最早可用优先”机制，采取一个基于规则的选择器，总是选择最快能够完成当前任务并且变得可用的电动车，这个机制的设置通过给每辆电动车设置一个“不可移动时间”，不可移动
    分两种情况，一种是移动到基站时，一种是移动到充电站时，同时不可移动时间会随着实际时间的流逝而线性减少
    2.节点选择器 ：Transformer-based model ：1.模型首先使用线性投影计算节点和ev的初始映射，嵌入计算 ：基站 ：输入基站状态包括2d坐标，容量，能量消耗率，t时刻的剩余电量，预计断电时间
                                        充电站 ：2d地理坐标，充电速率，是否正在被电动车访问的标志
                                        电动车 ：2d地理坐标，可用状态值
                                        2.堆叠L层编码器，每一层编码器允许所有节点和所有电动车进行信息交互，不使用位置编码，因为节点和电动车的访问顺序不影响最终结果
                                        3.节点访问概率计算 ：使用注意力机制计算相似度评分函数，根据softmax函数计算概率分布

    3.训练 ：使用reinforce算法进行策略梯度训练，将贪婪展开作为比较基准策略，采用保守的更新策略，

实验部分 ：
    1.数据集 训练流程是在合成数据集上面训练模型-》使用贪婪解码在验证集合上面选择成本最低的模型权重-》在独立评估集和真实数据集上面测试最终结果
    关键参数 ：电动车电池容量是60kwh；功耗率是161kwh/km;放电下限是0.1；放电速率是10kwh/h,行驶速度是41km/
    2.baseline : 随机节点选择器（RAND），对于选中的电动车，随机选择一个可访问节点作为下一个目的地，最终路线是12.8k采样路线中的最佳路线
                 贪婪节点选择（GREED），对选中的电动车，根据简单但是直观的规则选择下一个目的地，优先选择当前电量最低的可访问基站，如果没有可访问的基站，选择距离当前位置最近的充电站
                 CP-TSP：应用时空网络，沿着离散时间方向复制节点，使用弧来链接不同时间的节点
    3.超参数和设备 ：初始化方法：均匀分布，训练集1.28M，验证集1000，评价集100，训练设置：批次256，学习率10^-4,Transformer层数2，隐藏层维度128，注意力头数量8，权重因子：100
                    GPU ：1个a6000；CPU:2*P8380